In [11]:
# imports

import re
import pickle
import os

import pandas as pd
import matplotlib.pyplot as plt
from konlpy.tag import Okt

In [12]:
news_files=os.listdir('raw_data/naver_news_ranking_raw_data/')
print(len(set(sorted([i.split("_")[-1].split('.')[0] for i in news_files]))))

708


In [13]:
news_brands = list({i.split('_')[0] for i in news_files})

In [14]:
# read_pickles by news_brands
for news_brand in news_brands:
    pickle_files = [i for i in news_files if i.startswith(news_brand)]
    pickle_dates=[i.split('_')[-1].split('.')[0] for i in pickle_files] 
    vars()[news_brand] = dict()
    for pickle_date,pickle_file in zip(pickle_dates,pickle_files):
        with open(f'raw_data/naver_news_ranking_raw_data/{pickle_file}', 'rb') as f:
            vars()[news_brand][pickle_date] = pickle.load(f)

In [15]:
df_li = []
for news_brand in news_brands:
    temp = pd.DataFrame.from_dict(vars()[news_brand], orient='index')
    temp = pd.concat([temp.iloc[:,i] for i in range(len(temp.columns))])
    temp = pd.DataFrame(temp)    
    temp['news_brand'] = news_brand
    df_li.append(temp)

df_tot = pd.concat(df_li)

df_tot.rename(columns={0:'news_title_raw'},inplace=True)



In [6]:
def to_clean_str(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')    
    try:
        return hangul.sub('', text).strip()
    except:
        return ""
    
def sentence2word(text):
    okt = Okt()
    
    sentences_tag = []
    sentences_tag = okt.pos(text) 
    noun_list = []

    for word, tag in sentences_tag:
        if tag in ['Noun']: 
            noun_list.append(word)
    return ' '.join(noun_list)

In [8]:
df_tot

,news_title_raw,news_brand
20211104,12-year-old boy injured from a metal screw fou...,코리아헤럴드
20210313,"LH official takes life, land minister offers t...",코리아헤럴드
20220822,[Herald Interview] ‘Extraordinary Attorney Woo...,코리아헤럴드
20210510,Cases of ‘worrisome’ COVID-19 variants soar in...,코리아헤럴드
20211104,[News Focus] Elixir for diesel cars puts Korea...,코리아헤럴드
...,...,...
20220809,[단독] 서울 강남 빌딩 지하주차장에서 차량 확인하다 급류에 실종,매일경제
20220710,아베 사망에 '3일 동안 밀크티 1+1' 현수막 건 중국상점,매일경제
20220905,너바나 표지 속 ‘알몸 아기’가 낸 아동 포르노 소송 또 기각,매일경제
20210328,1010만원 솔비 13억 이우환…금융 IT 3040 미술품 쓸어담는다,매일경제


In [9]:
df_tot['news_title_cleaned']=df_tot['news_title_raw'].apply(lambda x:to_clean_str(x))
df_tot['news_words']=df_tot.news_title_cleaned.apply(lambda x: sentence2word(x))
df_tot.index=pd.to_datetime(df_tot.index,format="%Y%m%d")

In [10]:
df_tot.to_csv("preprocessed_data/news_data.csv")